In [ ]:
# import packages
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler, TargetEncoder
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, cross_val_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, roc_curve, roc_auc_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold

from utils import feature_engineering, additional_feature_engineering
# set the aesthetic style of the plots
sns.set_style()

# filter warning messages
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_credit = pd.read_csv('saved/preprocessed_bank_data.csv')
df_credit.info()


In [ ]:
X_processed = feature_engineering(df_credit)
X_processed.info()

In [ ]:
print(X_processed[X_processed['target_default'].isnull()])

In [ ]:
X_processed.nunique().sort_values()


In [ ]:
import pickle
import category_encoders as ce
X_processed["target_default"] = X_processed["target_default"].astype(int)

X = X_processed.drop(columns=["target_default"])
y = X_processed["target_default"]

categorical_cols = X.select_dtypes(exclude=['float64', 'int64']).columns.tolist()
# print(categorical_cols)
# # Check for missing values in each categorical column
# nan_counts = X_processed[categorical_cols].isnull().sum()

# # Print the columns with NaNs and their counts
# print(nan_counts[nan_counts > 0])
# Define the one-out encoder
oneout_encoder = ce.TargetEncoder(cols=categorical_cols)
# Fit and transform the categorical columns
X_processed[categorical_cols] = oneout_encoder.fit_transform(X[categorical_cols], y)



# Save label encoders to disk
with open('saved/label_encoders.pkl', 'wb') as f:
    pickle.dump(oneout_encoder, f)

In [ ]:
X_processed.nunique().sort_values()


In [ ]:
X_processed2 = additional_feature_engineering(X_processed)

In [ ]:
# import pandas as pd
# from imblearn.over_sampling import SMOTE


# # Separate features (X) and target (y)
# X = df_credit.drop('target_default', axis=1)
# y = df_credit['target_default']

# # Apply SMOTE
# smote = SMOTE(random_state=42)
# X_resampled, y_resampled = smote.fit_resample(X, y)

# # Create a new DataFrame with resampled data
# df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
# df_resampled['target_default'] = y_resampled

# Save the resampled data to a new CSV file
# df_resampled.to_csv('saved/feature_engineered_data.csv', index=False)

# print("SMOTE applied and saved to 'saved/feature_engineered_data.csv'")

In [ ]:
df_credit.to_csv('saved/feature_engineered_data.csv', index = False)